In [ ]:
import numpy as np
import pandas as pd
import sklearn
import cython
import talib
import bokeh

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from Functions import DataReady, KijunSen

In [ ]:
data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(data)

In [ ]:
class MachineLearningStrategyV2(Strategy):
    
    # Add variables to the strategy
    atr_lenght = 14
    times_atr_tp = 3
    times_atr_sl = 2
    upper_ema_lenght = 50
    lower_ema_lenght = 200
    stoch_rsi_lenght = 14
    kijun_sen_period = 26
    kijun_sen_price_range = 2
    # 15×5×5×50×50×15×20×10 = 2812500000

    def init(self):
        # Indicator code
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_lenght)
        self.ema_upper = self.I(talib.EMA, self.data.Close, self.upper_ema_lenght)
        self.ema_lower = self.I(talib.EMA, self.data.Close, self.lower_ema_lenght)
        self.stoch_rsi = self.I(talib.STOCHRSI, self.data.Close, self.stoch_rsi_lenght)
        self.kijun_sen = self.I(KijunSen, data, self.kijun_sen_period)
        self.kijun_sen_to_price = self.data.Close - self.kijun_sen

        
    def next(self):
        if crossover(self.ema_lower, self.ema_upper) and self.kijun_sen_to_price < self.kijun_sen_price_range / 10:
            self.buy(sl = self.data.Close - (self.atr*self.times_atr_sl), tp = self.data.Close + (self.atr*self.times_atr_tp))
        

    
bt = Backtest(data, MachineLearningStrategyV2, cash = 10000)

stats = bt.optimize(
    method = "skopt",
    upper_ema_lenght = range(20, 100, 5),
    lower_ema_lenght = range(100, 200, 5),
    kijun_sen_period = range(10, 40, 1),
    times_atr_tp = range(1, 5, 1),
    times_atr_sl = range(1, 5, 1),
    atr_lenght = range(5, 30, 1),
    kijun_sen_price_range = range(0, 10, 1),
    maximize = "Sharpe Ratio"
)

print(stats)

bt.plot()